## Load relevant packages

In [1]:
# Import all the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
from copy import deepcopy
%matplotlib inline

## Load relevant data files

In [2]:
#================================================================================================
# USER INPUT!
# Here I specify which data files need reading in
datafilenames = ["natinalitycleantrain.csv"]
# For each data file, we write which column contains the information we want to predict
topredictcolumn= ["Survived"]
#================================================================================================

# FROM HERE ON IT'S AUTOMATIC

# Sometiems the index column gets saved automatically as a new column when writingto csv. 
# We'll remove these as they serve no purpose.
alldataframes = [pd.read_csv(filename) for filename in datafilenames]

for (pos,frame) in enumerate(alldataframes):
    toadd = frame
    if "Unnamed: 0" in frame:
        toadd = toadd.drop("Unnamed: 0",axis=1)
    if "Unnamed: 0.1" in frame:
        toadd = toadd.drop("Unnamed: 0.1",axis=1)
    alldataframes[pos] = toadd

##### Still to do:
 - Make a function where you can input the xaxiscolumn and the yaxiscolumn and the type of regression (e.g. linear, logistic, etc.), and the function outputs the values of the regression parameters and the quality of the regression
 - Make a function that takes the data, a model name, what to predict and outputs predictions for the data and how well it did
 - Make a function that does clustering
 - Make a function that does feature engineering
 - Make plots to evaluate how well a certain prediction did (e.g. confusion matrices and ROC curves)
 - As helper-functions, might need to do bootstrapping functions etc., unless they're already in-built
 - Race models agains each other
 - Check what else we did during S2DS